In [9]:
"""
    tf1.0 自定义estimator
    
"""

'\n    tf1.0 estimator\n    使用预定义的estimator\n        BaselineClassifier\n        LinearClassifier\n        DNNClassifier\n    Tf.feature_column 做特征工程\n    \n    \n'

In [10]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import sklearn
import pandas as pd
import os
import sys
import time
import tensorflow as tf

from tensorflow import keras

print(tf.__version__)
print(sys.version_info)
for module in mpl, np, pd, sklearn, tf, keras:
    print(module.__name__, module.__version__)

1.13.1
sys.version_info(major=3, minor=6, micro=2, releaselevel='final', serial=0)
matplotlib 3.0.3
numpy 1.16.2
pandas 0.24.2
sklearn 0.20.3
tensorflow 1.13.1
tensorflow._api.v1.keras 2.2.4-tf


In [11]:
train_file = "./data/titanic/train.csv"
eval_file = "./data/titanic/eval.csv"

train_df = pd.read_csv(train_file)
eval_df = pd.read_csv(eval_file)

y_train = train_df.pop("survived")
y_eval =  eval_df.pop("survived")

print(train_df.head(2))
print(eval_df.head(2))
print(y_train.head(2))
print(y_eval.head(2))


      sex   age  n_siblings_spouses  parch     fare  class     deck  \
0    male  22.0                   1      0   7.2500  Third  unknown   
1  female  38.0                   1      0  71.2833  First        C   

   embark_town alone  
0  Southampton     n  
1    Cherbourg     n  
    sex   age  n_siblings_spouses  parch     fare  class     deck  \
0  male  35.0                   0      0   8.0500  Third  unknown   
1  male  54.0                   0      0  51.8625  First        E   

   embark_town alone  
0  Southampton     y  
1  Southampton     y  
0    0
1    1
Name: survived, dtype: int64
0    0
1    0
Name: survived, dtype: int64


In [12]:
# feature_column
categorical_columns = ["sex", "n_siblings_spouses", "parch", 
                       "class", "deck", "embark_town", "alone"]
numeric_columns = ["age","fare" ]
feature_columns = []
for categorical_column in categorical_columns:
    vocab = train_df[categorical_column].unique()
    # 离散特征作成one-hot编码
    onehot_describe = tf.feature_column.indicator_column(
        tf.feature_column.categorical_column_with_vocabulary_list(
            categorical_column, vocab))
    feature_columns.append(onehot_describe)
    
for numeric_column in numeric_columns:
    feature_columns.append(tf.feature_column.numeric_column(numeric_column, dtype = tf.float32))

In [13]:
def make_dataset(data_df, label_df, epochs = 10,# 数据集遍历多少次 ， 
                 shuffle = True, batch_size = 32):
    dataset = tf.data.Dataset.from_tensor_slices(
         (dict(data_df),label_df))
    if shuffle:
        dataset = dataset.shuffle(10000)
    dataset = dataset.repeat(epochs).batch(batch_size)
    return dataset.make_one_shot_iterator().get_next()

In [19]:
output_dir = "customized_estimator"
if not os.path.exists(output_dir):
    os.mkdir(output_dir)
def model_fn(features, labels, mode, params):
    # model runtime state: train; eval; predict
    input_for_next_layer = tf.feature_column.input_layer(
        features, params["feature_column"])
    for n_unit in params["hidden_units"]:
        input_for_next_layer = tf.layers.dense(input_for_next_layer,
                                              units = n_unit,
                                              activation = tf.nn.relu)
    logits = tf.layers.dense(input_for_next_layer,
                            params["n_classes"],
                            activation = None)  
    predicted_classes = tf.argmax(logits, 1)
    
    if mode == tf.estimator.ModeKeys.PREDICT:
        predictions = {
            "class_ids":predicted_classes[:, tf.newaxis],
            "probabilities": tf.nn.softmax(logits),
            "logits":logits
        }
        return tf.estimator.EstimatorSpec(mode,
                                      predictions = predictions)
    loss = tf.losses.sparse_softmax_cross_entropy(labels = labels,
                                                 logits = logits)
    accuracy = tf.metrics.accuracy(labels = labels, 
                                   predictions = predicted_classes,
                                   name = "acc_op")
    metrics = {"accuracy":accuracy}
    
    if mode == tf.estimator.ModeKeys.EVAL:
        return tf.estimator.EstimatorSpec(mode, loss = loss,
                                          eval_metric_ops=metrics)
    
    optimizer = tf.train.AdamOptimizer()
    train_op = optimizer.minimize(loss, 
                                  global_step = tf.train.get_global_step())
    if mode == tf.estimator.ModeKeys.TRAIN:
        return tf.estimator.EstimatorSpec(mode, loss = loss,
                                          train_op = train_op)

estimator = tf.estimator.Estimator(
    model_fn = model_fn, 
    model_dir=output_dir, 
    params= {
        "feature_column": feature_columns,
        "hidden_units": [100,100],
        "n_classes":2
    }
)

estimator.train(input_fn = lambda : make_dataset(
    train_df, y_train, epochs = 100))

INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_model_dir': 'customized_estimator', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000001616B079BE0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:

In [20]:
estimator.evaluate(lambda:make_dataset(eval_df, y_eval, epochs = 1))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2020-02-09T14:52:05Z
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from customized_estimator\model.ckpt-1960
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2020-02-09-14:52:05
INFO:tensorflow:Saving dict for global step 1960: accuracy = 0.79545456, global_step = 1960, loss = 0.49870354
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 1960: customized_estimator\model.ckpt-1960


{'accuracy': 0.79545456, 'global_step': 1960, 'loss': 0.49870354}